# SiRIUS tsys-manual noise calculation

The following explanation is partially based on pages 188-192 in Thompson et al.'s second edition of  Interferometry and Synthesis in Radio Astronomy.

The <b>system temperature (K)</b> (with the zenith atmospheric opacity $\tau=0$)* is:
\begin{align}
T_{sys} &= T_{rx} + T_{atmos}(1 - \eta_{spill} ) + T_{cmb}\\
\end{align}
where
 - $T_{rx}$ is the receiver temperature ($K$),
 - $T_{atmos}$ is the atmospheric temperature ($K$),
 - $\eta_{spill}$ is the forward spillover efficiency,
 - $T_{cmb}$ is the cosmic microwave background temperature ($K$).
*In a future release the atmospheric opacity will be included.

The <b>effective dish area ($m^2$)</b> for baseline $ij$:
\begin{align}
A^{eff}_{ij} = \eta_a \frac{\pi d_i d_j}{4}
\end{align}
where
 - $\eta_a$ is the antenna efficiency,
 - $d_{i/j}$ is the dish diamter of the $i/j$ antenna.

The <b>rms noise level (Jy)</b> at the correlator for the $ij$ baseline for a single component of the visibility (real or complex):
\begin{align}
\sigma_{ij} = \frac{\sqrt{2} k_b T_{sys} × 10^{26}}{A^{eff}_{i,j} \eta_q \eta_{cor} \sqrt{\Delta t \Delta \nu}}
\end{align}
where
- $k_b = 1.38064852 × 10^{-23} m^2 \: kg \: s^{-2} K^{-1}$ is Boltzmann's constant,
- $10^{26}$ is the factor that converts from $J m^{-2}$ to $Jy$.
- $\eta_q$ is the digitizer quantization efficiency,
- $\eta_{cor}$ is the correlator efficiency,
- $\Delta t$ is the correlator dump time (Seconds),
- $\Delta \nu$ is the bandwidth of a single channel (Hertz).

SiRIUS will save the following values to the main table of the measuremnet set (version 2):
\begin{align}
SIGMA_{ij} &= \sigma_{ij}, \\
WEIGHT_{ij} &= 1/\sigma_{ij}^2.
\end{align}

In the synthesized map, with natural weighting, the flux density of a point source for which the peak response is equal to the rms noise level is given by:
\begin{align}
S_{rms} = \sqrt{1/\left( \sum^{n_{t}} \sum^{n_{b}} \sum^{n_{chan}} \sum^{n_{pol}} \frac{1}{\sigma_{ij}^2} \right)} 
\end{align}
where
- $n_{t}$ is the number of time samples with dump time $\Delta t$,
- $n_{b}$ is the number of baselines,
- $n_{chan}$ number of channels with bandwidth $\Delta \nu$,
- $n_{pol}$ number of instrumental polarizations used to create the image (for stokes I, $n_{pol} = 2$).

If $d=d_i = d_j$ ($\epsilon_{ij}$ is no longer a function baseline) we can simplify the point source rms sensitivity to:
\begin{align}
S_{nat} = \frac{\epsilon_{ij}}{\sqrt{n_{t} n_{b} n_{chan}n_{pol}}} = \frac{\sqrt{2} k_b T_{sys} × 10^{26}}{A^{eff} \eta_q \eta_{cor} \sqrt{ (n_t \Delta t) n_b (n_{chan} \Delta \nu) n_{pol}}} = \frac{\sqrt{2} k_b T_{sys} × 10^{26}}{A^{eff} \eta_q \eta_{cor} \sqrt{ t_{int} \nu_{int} n_b n_{pol}}} = \frac{2 k_b T_{sys} × 10^{26}}{A^{eff} \eta_q \eta_{cor} \sqrt{ t_{int} \nu_{int} n_a (n_a - 1) n_{pol}}}
\end{align}

when a weighting scheme is added:
\begin{align}
S_{wgt} = \frac{S_{nat}}{\eta_{wgt}} = S_{nat} \frac{\sqrt{\sum w_i^2}}{\sum w_i}.
\end{align}

The <b>antenna system equivalent flux density ($Jy$)</b>:
\begin{align}
SEFD &= \frac{2 k_b T_{sys}}{\eta_q A^{eff}} × 10^{26} \\
\end{align}

\begin{align}
S_{wgt} &= \frac{ SEFD }{\eta_{wgt} \eta_{cor} \sqrt{ t_{int} \nu_{int} n_a (n_a - 1) n_{pol}}} .
\end{align}

\begin{align}
t_{int} =  \frac{t_{int,nat}}{\eta_{wgt}^2} = \frac{n_{wgt}^2 SEFD^2}{\eta_{wgt}^2 S_{wgt}^2 \eta^2_{cor} \nu_{int} n_a (n_a - 1) n_{pol}}
\end{align}


# Visibility Averaging Time

\begin{align}
\Delta \tau \approx \frac{C}{\omega_l b_{\lambda} \theta_f}
\end{align}
where

- $\Delta \tau$  (seconds)
- $C$ fractor (0.1-0.5)
- $\omega_l$ angular velocity of the earth
- $b_{\lambda}$ basline length in wavelengths
- $\theta_f$ field of view


In [1]:
import pkg_resources
import xarray as xr
import numpy as np
import dask
from astropy.coordinates import SkyCoord

tel_dir = pkg_resources.resource_filename('sirius_data', 'telescope_layout/data/ngvla-revD.main.tel.zarr')
print(tel_dir)
tel_xds = xr.open_zarr(tel_dir,consolidated=False)
n_ant = tel_xds.dims['ant_name']

tel_xds

/users/jsteeb/sirius/sirius_data/telescope_layout/data/ngvla-revD.main.tel.zarr


<xarray.Dataset>
Dimensions:        (ant_name: 214, pos_coord: 3)
Coordinates:
  * ant_name       (ant_name) <U6 'mda001' 'mda002' ... 'cor113' 'cor114'
  * pos_coord      (pos_coord) int64 0 1 2
Data variables:
    ANT_POS        (ant_name, pos_coord) float64 dask.array<chunksize=(214, 3), meta=np.ndarray>
    DISH_DIAMETER  (ant_name) float64 dask.array<chunksize=(214,), meta=np.ndarray>
Attributes:
    site_pos:        [{'m0': {'unit': 'm', 'value': -1601156.673287362}, 'm1'...
    telescope_name:  NGVLA

In [2]:
a = tel_xds.ANT_POS[0:2,:].values
print(a)
np.sqrt(np.sum((a[0,:] - a[1,:])**2))

[[-1579550.16218 -5053312.5996   3548420.59509]
 [-1576468.20511 -5074253.15704  3519418.81076]]


35904.16270577955

# SiRIUS tsys-manual noise calculation (bu)

The following explanation is partially based on pages 188-192 in Thompson et al.'s second edition of  Interferometry and Synthesis in Radio Astronomy.

The <b>system temperature (K)</b> (with the zenith atmospheric opacity $\tau=0$)* is:
\begin{align}
T_{sys} &= T_{rx} + T_{atmos}(1 - \eta_{spill} ) + T_{cmb}\\
\end{align}
where
 - $T_{rx}$ is the receiver temperature ($K$),
 - $T_{atmos}$ is the atmospheric temperature ($K$),
 - $\eta_{spill}$ is the forward spillover efficiency,
 - $T_{cmb}$ is the cosmic microwave background temperature ($K$).
*In a future release the atmospheric opacity will be included.

The <b>effective dish area ($m^2$)</b> for baseline $ij$:
\begin{align}
A^{eff}_{ij} = \eta_a \frac{\pi d_i d_j}{4}
\end{align}
where
 - $\eta_a$ is the antenna efficiency,
 - $d_{i/j}$ is the dish diamter of the $i/j$ antenna.

The <b>rms noise level (Jy)</b> at the correlator for the $ij$ baseline for a single component of the visibility (real or complex):
\begin{align}
\sigma_{ij} = \frac{\sqrt{2} k_b T_{sys} × 10^{26}}{A^{eff}_{i,j} \eta_q \eta_{cor} \sqrt{\Delta t \Delta \nu}}
\end{align}
where
- $k_b = 1.38064852 × 10^{-23} m^2 \: kg \: s^{-2} K^{-1}$ is Boltzmann's constant,
- $10^{26}$ is the factor that converts from $J m^{-2}$ to $Jy$.
- $\eta_q$ is the digitizer quantization efficiency,
- $\eta_{cor}$ is the correlator efficiency,
- $\Delta t$ is the correlator dump time (Seconds),
- $\Delta \nu$ is the bandwidth of a single channel (Hertz).

By combining the real and imaginary outputs of the correlator the <b>rms uncertainty</b> in the measurement is:
\begin{align}
\epsilon_{ij} = \sqrt{2} \sigma_{ij} = \frac{2 k_b T_{sys} × 10^{26}}{A^{eff}_{i,j} \eta_q \eta_{cor} \sqrt{\Delta t \Delta \nu}}.
\end{align}
SiRIUS will save the following values to the main table of the measuremnet set (version 2):
\begin{align}
SIGMA_{ij} &= \epsilon_{ij}, \\
WEIGHT_{ij} &= 1/\epsilon_{ij}^2.
\end{align}

In the synthesized map, with natural weighting, the flux density of a point source for which the peak response is equal to the rms noise level is given by:
\begin{align}
S_{rms} = \sqrt{1/\left( \sum^{n_{t}} \sum^{n_{b}} \sum^{n_{chan}} \sum^{n_{pol}} \frac{1}{\epsilon_{ij}^2} \right)} 
\end{align}
where
- $n_{t}$ is the number of time samples with dump time $\Delta t$,
- $n_{b}$ is the number of baselines,
- $n_{chan}$ number of channels with bandwidth $\Delta \nu$,
- $n_{pol}$ number of instrumental polarizations used to create the image (for stokes I, $n_{pol} = 2$).

If $d=d_i = d_j$ ($\epsilon_{ij}$ is no longer a function baseline) we can simplify the point source rms sensitivity to:
\begin{align}
S_{nat} = \frac{\epsilon_{ij}}{\sqrt{n_{t} n_{b} n_{chan}n_{pol}}} = \frac{2 k_b T_{sys} × 10^{26}}{A^{eff} \eta_q \eta_{cor} \sqrt{ (n_t \Delta t) n_b (n_{chan} \Delta \nu) n_{pol}}} = \frac{2 k_b T_{sys} × 10^{26}}{A^{eff} \eta_q \eta_{cor} \sqrt{ t_{int} \nu_{int} n_b n_{pol}}}
\end{align}

when a weighting scheme is added:
\begin{align}
S_{wgt} = \frac{S_{nat}}{\eta_{wgt}} = S_{nat} \frac{\sqrt{\sum w_i^2}}{\sum w_i}.
\end{align}

The <b>antenna system equivalent flux density ($Jy$)</b>:
\begin{align}
SEFD &= \frac{2 k_b T_{sys}}{\eta_q A^{eff}} × 10^{26} \\
\end{align}

\begin{align}
S_{wgt} &= \frac{ SEFD }{\eta_{wgt} \eta_{cor} \sqrt{ t_{int} \nu_{int} n_b n_{pol}}} .
\end{align}

\begin{align}
t_{int} =  \frac{t_{int,nat}}{\eta_{wgt}^2} = \frac{n_{wgt}^2 SEFD^2}{\eta_{wgt}^2 S_{wgt}^2 \eta^2_{cor} \nu_{int} n_b n_{pol}}
\end{align}




The following source have information about the ngVLA system parameters that are needed to calculate on the on source time:

Resources:
- ngVLA Performance Estimates https://ngvla.nrao.edu/page/performance (2021-12)
- System Requirements, pages 91 to 99 (https://ngvla.nrao.edu/download/MediaFile/254/original (2020-05-04).
- System Reference Design, pages 18-19 https://ngvla.nrao.edu/download/MediaFile/226/original (2019-07-25)
- ngVLA Sensitivity https://library.nrao.edu/public/memos/ngvla/NGVLA_21.pdf (2019-06-27)
- Science Requirements https://ngvla.nrao.edu/download/MediaFile/248/original (2019-11-19)
 
Noe that the system parameters are not identical between the documents. The most recent documents will be given preference.


Subject to change
Naturally weighted

Need for equations to display correctly https://jupyter-contrib-nbextensions.readthedocs.io/en/latest/install.html

We are going simplify this. Purpose is to get rough estimate of the required total observation time. Assume all antennas have same diameter (no center antennas).


\begin{align}
SEFD &= \frac{2 k_b T_{sys}}{\eta_q \eta_a A} × 10^{26} \\
\end{align}

where

-  SEFD is the antenna system equivalent flux density ($Jy$),
- $k_b = 1.38064852 × 10^{-23} m^2 \: kg \: s^{-2} K^{-1}$ is Boltzmann's constant,
- $T_{sys}$ is the system tempreture ($K$),
- $\eta_q$ is the digitizer quantization efficiency,
- $\eta_a$ is the antenna efficiency.
- $A = \frac{\pi d^2}{4}$ is the antenna’s geometric collecting area ($m^2$) where d is the dish diamter ($m$),
- $10^{26}$ is the factor that converts from $J m^{-2}$ to $Jy$.

The weighted point source rms sensitivity is computed as: 
$$
\sigma_{rms} = \eta_{wgt} \sigma_{na} = \eta_{wgt} \frac{SEFD}{\eta_{cor} \sqrt{n_{pol}n_{ant}(n_{ant}-1)t_{int}\Delta \nu}} \label{eq1}
$$
where
- $\sigma_{na}$ naturally weighted point source rms sensitivity (Jy),
- $\eta_{wgt}$
- $\eta_{cor}$ is the correlator efficiency,
- $n_{pol}$ is the number of polarization products included in the image; npol = 2 for images in Stokes I, Q, U, or V, and npol = 1 for images in RCP or LCP,
- $n_{ant}$ is the number of antennas,
- $t_{int}$ The total on source integration time ($s$),
- $\Delta \nu$ bandwidth (Hz).


$$
t_{int} = \frac{SEFD^2}{\eta_{wgt}^2\sigma_{na}^2 \eta_{cor}^2 n_{pol}n_{ant}(n_{ant}-1)\Delta \nu } \label{eq2}
$$





In [3]:
import numpy as np
import scipy.constants as spc


'''
import xarray as xr
coords = {'band':['B1','B2','B3','B4','B5','B6'],'f_center':('band',np.array([2.4,8,16,27,41,93])*10**9)}
ng_xds = xr.Dataset()
ng_xds = ng_xds.assign_coords(coords)
ng_xds['eta_a'] = xr.DataArray(np.array([0.828,0.936,0.941,0.920,0.886,0.648]), dims=['band'])
ng_xds['T_sys'] = xr.DataArray(np.array([17.07,22.00,24.40,32.42,47.41,65.37]), dims=['band'])
ng_xds['max_bw'] = xr.DataArray(np.array([2.3,8.8,8.2,13.5,20.0,20.0])*10**9, dims=['band'])
ng_xds.attrs['eta_q'] = 0.96
ng_xds.attrs['eta_c'] = 0.99
ng_xds.attrs['n_pol'] = 2
ng_xds.attrs['d'] = 18
ng_xds.attrs['n_ant'] = 214
print(ng_xds)
'''

eta_a = np.array([0.828,0.936,0.941,0.920,0.886,0.648])
T_sys = np.array([17.07,22.00,24.40,32.42,47.41,65.37])
max_bw = np.array([2.3,8.8,8.2,13.5,20.0,20.0])*10**9
eta_q = 0.96
eta_cor = 0.99
n_pol = 2
d = 18
n_ant = 214



In [4]:
def calc_SEFD(T_sys,eta_q,eta_a,d):
    A = np.pi*d**2/4
    k_B = spc.k
    return 2*k_B*T_sys/(eta_a*eta_q*A)*10**26

def calc_sigma_na(SEFD,eta_cor,n_pol,n_ant,t_int,delta_nu):
    return SEFD/(eta_cor*np.sqrt(n_pol*n_ant*(n_ant-1)*t_int*delta_nu))

def calc_t_int(SEFD,sigma_na,eta_cor,n_pol,n_ant,delta_nu):
    return (SEFD**2)/(sigma_na**2*eta_cor**2*n_pol*n_ant*(n_ant-1)*delta_nu)

In [5]:
SEFD = calc_SEFD(T_sys,eta_q,eta_a,d)
print(SEFD)

[233.02919669432143 265.67701664487095 293.09428929292926
 398.32020510526723 604.8440143725015 1140.2777046211004]


## Continuum

Science requirements in page 26 of https://ngvla.nrao.edu/download/MediaFile/248/original and cases in page 91 of https://ngvla.nrao.edu/download/MediaFile/254/original.

### Case A

In [6]:
sigma_na = 0.07*10**(-6)    # Jy/beam, naturally weighted point source rms sensitivity
delta_nu = 13.5*10**9       # Hz, Instantaneous bandwidth
band_indx = 3

t_int = calc_t_int(SEFD[band_indx],sigma_na*0.5,eta_cor,n_pol,n_ant,max_bw[band_indx])/3600
print(t_int)

29.82624316263078


### Case B

In [7]:
sigma_na = 0.5*10**(-6)    # Jy/beam, naturally weighted point source rms sensitivity
band_indx = 5

t_int = calc_t_int(SEFD[band_indx],sigma_na*0.5,eta_cor,n_pol,n_ant,max_bw[band_indx])/3600
print(t_int)

3.2338136526508157
